In [6]:
from nltk import tokenize
import sys
import codecs
import spacy
import re
nlp = spacy.load('en')



def read_lines(fname):
    ret=[]
    indx=-1
    for line in codecs.open(fname, encoding="utf-8"):
        sent = line
        sent = sent.replace("-LRB-","(")
        sent = sent.replace("-RRB-",")")
        sent = sent.replace('\\n',' ')
        sent = sent.replace('\n',' ')
        sent = re.sub(r"\\x[a-z,A-Z,0-9][a-z,A-Z,0-9]",'',sent)
        #sent = unicodetoascii(sent)
        #sent = clearchinesetext(sent)
        indx=indx+1
        ret.append(sent)
    
    return ret

def clearchinesetext(sent):
    SENT = [word for word in sent if any(letter in allowed for letter in word)]
    print SENT
    return SENT

In [9]:
content = read_lines("/Users/vmangipudi/EA/output_gensim.txt")
sents=tokenize.sent_tokenize(''.join(content))    
sents


from collections import defaultdict
# remember to include the other import from the previous post
from nltk.corpus import stopwords
import string
 
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via'] 
com = defaultdict(lambda : defaultdict(int))
 
# f is the file pointer to the JSON data set
for line in sents: 
    terms_only = [word for word in sents
                if word not in stop 
                and not word.startswith(('#', '@'))]
 
    # Build co-occurrence matrix
    for i in range(len(terms_only)-1):            
        for j in range(i+1, len(terms_only)):
            w1, w2 = sorted([terms_only[i], terms_only[j]])                
            if w1 != w2:
                com[w1][w2] += 1

com_max = []
# For each term, look for the most common co-occurrent terms
for t1 in com:
    t1_max_terms = sorted(com[t1].items(), key=operator.itemgetter(1), reverse=True)[:5]
    for t2, t2_count in t1_max_terms:
        com_max.append(((t1, t2), t2_count))
# Get the most frequent co-occurrences
terms_max = sorted(com_max, key=operator.itemgetter(1), reverse=True)
print(terms_max[:5])

NameError: name 'operator' is not defined

In [91]:
import itertools

class entityClass(object):

    def __init__(self, name, type,start,end):
        self.name = name
        self.type = type
        self.start = start
        self.end = end

    def getName(self):
        return self.name

    def getType(self):
        return self.type
    
    def getStart(self):
        return self.start

    def getEnd(self):
        return self.end

    def __str__(self):
        return "%s is a %s" % (self.name, self.type)


entities=[]
interactions=[]
entity_dict=dict()
to_gensim=[]
for sent in sents:
    sent = nlp(sent)
    #print sent.ents
    for ent in sent.ents:
            if ent.label_ in ["PERSON","ORG","LOC","GPE"] and ent.text.strip() not in ["the","a"]:
                entity_dict[ent.text.strip()]=ent.label_
        
        
#for d in entity_dict: print d
        
for sent in sents:
    sent = nlp(sent)
    person_found=0
    location_found=0
    email_found=0
    org_found=0
    gpe_found=0
    url_found=0
    ip_found=0
    word_in_dict_found=0
    person_ents=[]
    org_ents=[]
    loc_ents=[]
    gpe_ents=[]
    email_ents=[]
    url_ents=[]
    ip_ents=[]
    url_ents=[]
    allents=[]
    allents_str=[]
    
    for ent in sent.ents:
        if ent.label_ in ["PERSON"]:
            e=entityClass(ent.text.strip(),"PERSON",ent.start_char,ent.end_char)
            person_ents.append(e)
            person_found=person_found+1
    
    for ent in sent.ents:
        if ent.label_ in ["ORG"]:
            e=entityClass(ent.text.strip(),"ORG",ent.start_char,ent.end_char)
            org_ents.append(e)
            org_found=org_found+1
    
    for ent in sent.ents:
        if ent.label_ in ["LOC"]:
            e=entityClass(ent.text.strip(),"LOC",ent.start_char,ent.end_char)
            loc_ents.append(e)
            location_found=location_found+1
    
    for ent in sent.ents:
        if ent.label_ in ["GPE"]:
            e=entityClass(ent.text.strip(),"GPE",ent.start_char,ent.end_char)
            gpe_ents.append(e)
            gpe_found=gpe_found+1
    
    
    for word in sent:
        if word.like_url==True:
            e=entityClass(word.text.strip(),"URL",word.idx,word.idx + word.__len__())
            url_ents.append(e)
            url_found=url_found+1
            
    for word in sent:      
        if re.search(r'[\w\.-]+@[\w\.-]+',word.text):
            print word.text
            e=entityClass(word.text.strip(),"EMAIL",word.idx,word.idx + word.__len__())
            email_ents.append(e)
            email_found=email_found+1

    for word in sent:      
        if re.search(r'[0-9]+(?:\.[0-9]+){3}',word.text):
            #print word.text
            e=entityClass(word.text.strip(),"IP",word.idx,word.idx + word.__len__())
            ip_ents.append(e)
            ip_found=ip_found+1    
    
    
    allents = org_ents+email_ents+loc_ents+ip_ents+person_ents+url_ents
    for e in allents:allents_str.append(e.name)
    
    #for e in allents:print e
    #for e in allents_str:print e
    #catch all entities missed by POS tagger
    for word in sent:
        if not word.text.strip() in allents_str and word.text in entity_dict:
            word_in_dict_found=word_in_dict_found+1
            e=entityClass(word.text.strip(),entity_dict[word.text],word.idx,word.idx + word.__len__())
            allents.append(e)
    
    sent_tx=sent.text
    for ent in allents: 
        if ent.name.find(" ") > -1:
            print ent.name
            sent_tx = sent_tx.replace(ent.name,ent.name.replace(" ","_"),1)
    
    to_gensim.append(sent_tx)
    
    
    
    cp_allents=[]
    cp_allents_str=[]

    for e in allents:
        if e.name.strip() not in ["the","a","APT1"]:
            cp_allents.append(e)
            cp_allents_str.append(e.name)
    
    allents=cp_allents;allents_str=cp_allents_str;

    cp_allents=[]
    cp_allents_str=[]
    for e in allents:
        if not e.name.strip() in cp_allents_str:
            cp_allents.append(e)
            cp_allents_str.append(e.name)
        
    allents=cp_allents;allents_str=cp_allents_str;
    
    for e in itertools.combinations(allents, 2):
        (a,b) =e
        #print (a,b)
        entities.append(e)
     
    if len(allents) > 2:
        print sent
    #heuristics to detemine strength of entity relation,weights 1,2,3 
    #for ent in allents: print ent
    ########Commeneted##########
#    for (a,b) in itertools.combinations(allents, 2):
#        print (a.name,a.start,b.name,b.start)
#        if(a.start>b.start): c=b;b=a;a=c;
#            #print sent[a.end:b.start]
#        if(b.start - a.end < 100) and any (wrd in sent[a.end:b.start].text for wrd in ["located","situated","email address","IP address"] ):
#            print (a.name,a.type,b.name,b.type)
#            #print sent[a.end:b.start].text
#        if (b.start - a.end < 20) and any (wrd in sent[a.end:b.start].text for wrd in ["is not","not belong","not member"]):
#            print (a.name,a.type,b.name,b.type)
            
            #print sent[b.start:a.end]
        #print (a.name,a.type,b.name,b.type,a.start,a.end,b.start,b.end)
        
        #####################################

import gensim, logging
import nltk
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
tok_corp=[nltk.word_tokenize(line) for line in to_gensim]
model = gensim.models.Word2Vec(tok_corp,min_count=1,size=200,sample = 1e-2)        
        
#print interactions

for (a,b) in entities:
    try:
        mod_a=a.name.replace(" ","_")
        mod_b=b.name.replace(" ","_")
        interactions.append((a.name,a.type,b.name,b.type,model.similarity(mod_a,mod_b)))
    except Exception as e:
        #print e
        interactions.append((a.name,a.type,b.name,b.type,.5))
    
    
#print interactions    

Chinas Computer Network Operations Tasking
Mandiant Distinguish
'APT1 Exposing One of Chinas Cyber  Espionage Units Contents Executive Summary .......................................................................................................... 2 Chinas Computer Network Operations Tasking to PLA Unit 61398 (61398) .................... 7 APT1: Years of Espionage  ............................................................................................... 20 APT1: Attack Lifecycle .................................................................................................... 27 APT1: Infrastructure ....................................................................................................... 39 APT1: Identities ............................................................................................................. 51 Conclusion ..................................................................................................................... 59 Appendix A: Ho

In [76]:
print interactions

[(u'Chinas Computer Network Operations Tasking', 'ORG', u'Lifecycle', 'PERSON', -0.67623574955789278), (u'Chinas Computer Network Operations Tasking', 'ORG', u'Mandiant Distinguish', 'PERSON', -0.67219777630494382), (u'Chinas Computer Network Operations Tasking', 'ORG', u'Chinas', u'GPE', -0.69152719538404261), (u'Chinas Computer Network Operations Tasking', 'ORG', u'Computer', u'ORG', -0.68414399493558098), (u'Chinas Computer Network Operations Tasking', 'ORG', u'Operations', u'ORG', -0.69017375664366254), (u'Chinas Computer Network Operations Tasking', 'ORG', u'PLA', u'ORG', -0.68906178877119617), (u'Chinas Computer Network Operations Tasking', 'ORG', u'Unit', u'PERSON', -0.68762073224047504), (u'Chinas Computer Network Operations Tasking', 'ORG', u'61398', u'PERSON', -0.68588848551759329), (u'Chinas Computer Network Operations Tasking', 'ORG', u'7', u'ORG', -0.69255459371115036), (u'Chinas Computer Network Operations Tasking', 'ORG', u'Appendix', u'PERSON', -0.68747014494200298), (u

In [77]:
model.most_similar("Harbin_Institute_of_Technology")

[(u'61398', 0.9941452741622925),
 (u'Datong', 0.994043231010437),
 (u'3', 0.9938942193984985),
 (u'General', 0.9938175678253174),
 (u'2', 0.9937269687652588),
 (u'Center', 0.9937133193016052),
 (u'6', 0.9936614036560059),
 (u'2010', 0.9936220645904541),
 (u'January', 0.9936137795448303),
 (u'2013', 0.9935953617095947)]

email_ents

In [78]:
print interactions

[(u'Chinas Computer Network Operations Tasking', 'ORG', u'Lifecycle', 'PERSON', -0.67623574955789278), (u'Chinas Computer Network Operations Tasking', 'ORG', u'Mandiant Distinguish', 'PERSON', -0.67219777630494382), (u'Chinas Computer Network Operations Tasking', 'ORG', u'Chinas', u'GPE', -0.69152719538404261), (u'Chinas Computer Network Operations Tasking', 'ORG', u'Computer', u'ORG', -0.68414399493558098), (u'Chinas Computer Network Operations Tasking', 'ORG', u'Operations', u'ORG', -0.69017375664366254), (u'Chinas Computer Network Operations Tasking', 'ORG', u'PLA', u'ORG', -0.68906178877119617), (u'Chinas Computer Network Operations Tasking', 'ORG', u'Unit', u'PERSON', -0.68762073224047504), (u'Chinas Computer Network Operations Tasking', 'ORG', u'61398', u'PERSON', -0.68588848551759329), (u'Chinas Computer Network Operations Tasking', 'ORG', u'7', u'ORG', -0.69255459371115036), (u'Chinas Computer Network Operations Tasking', 'ORG', u'Appendix', u'PERSON', -0.68747014494200298), (u

In [79]:
#for np in sent.noun_chunks:
#    print np

for ent in sent.ents:
    print ent
    #print ent.start
    #print sent[ent.start+1]
    #print ent.label_


74


In [80]:
import networkx as nx
import matplotlib.pyplot as plt
color_map=dict()
color_map["ORG"]={'color': {'r': 0, 'g': 0, 'b': 255, 'a': 0}}#blue
color_map["GPE"]={'color': {'r': 0, 'g': 255, 'b': 0, 'a': 0}}#green
color_map["PERSON"]={'color': {'r': 255, 'g': 0, 'b': 0, 'a': 0}}#red
color_map["IP"]={'color': {'r': 255, 'g': 165, 'b': 0, 'a': 0}}#magenta?
color_map["URL"]={'color': {'r': 255, 'g': 255, 'b': 0, 'a': 0}}#yellow
color_map["LOC"]={'color': {'r': 255, 'g': 0, 'b': 255, 'a': 0}}
color_map["EMAIL"]={'color': {'r': 0, 'g': 0, 'b': 0, 'a': 0}}
%matplotlib inline
group_attr=[]
node_dict=dict()
attr_dict=dict()
G=nx.Graph()
pos=nx.spring_layout(G)
e2e_interactions=[]
for i in interactions:
    group_attr.append(tuple([i[0],i[2]]))
    node_dict[i[0]]=color_map[i[1]]
    node_dict[i[2]]=color_map[i[3]]
    attr_dict[i[0]]=i[1]
    attr_dict[i[2]]=i[3]
    G.add_edge(i[0],i[2],weight=i[4])


    
#nx.draw_networkx(G,nodelist=list(node_dict.keys()),node_color=list(node_dict.values()))
#nx.write_gexf(G, "/Users/vmangipudi/test.gexf")
#nx.draw_networkx_nodes(G,list(node_dict.keys()),list(node_dict.values()))

In [81]:
for n in node_dict:
    print node_dict[n]
    G.node[n]['viz']=node_dict[n]
    G.node[n]['attr']=attr_dict[n]
nx.write_gexf(G, "/Users/vmangipudi/test6_noapt.gexf")
    

{'color': {'a': 0, 'r': 0, 'b': 0, 'g': 255}}
{'color': {'a': 0, 'r': 0, 'b': 255, 'g': 0}}
{'color': {'a': 0, 'r': 0, 'b': 255, 'g': 0}}
{'color': {'a': 0, 'r': 0, 'b': 255, 'g': 0}}
{'color': {'a': 0, 'r': 0, 'b': 255, 'g': 0}}
{'color': {'a': 0, 'r': 0, 'b': 255, 'g': 0}}
{'color': {'a': 0, 'r': 0, 'b': 255, 'g': 0}}
{'color': {'a': 0, 'r': 0, 'b': 255, 'g': 0}}
{'color': {'a': 0, 'r': 0, 'b': 255, 'g': 0}}
{'color': {'a': 0, 'r': 0, 'b': 0, 'g': 0}}
{'color': {'a': 0, 'r': 255, 'b': 0, 'g': 255}}
{'color': {'a': 0, 'r': 0, 'b': 255, 'g': 0}}
{'color': {'a': 0, 'r': 255, 'b': 0, 'g': 255}}
{'color': {'a': 0, 'r': 255, 'b': 0, 'g': 255}}
{'color': {'a': 0, 'r': 0, 'b': 0, 'g': 255}}
{'color': {'a': 0, 'r': 0, 'b': 0, 'g': 255}}
{'color': {'a': 0, 'r': 0, 'b': 255, 'g': 0}}
{'color': {'a': 0, 'r': 0, 'b': 255, 'g': 0}}
{'color': {'a': 0, 'r': 0, 'b': 255, 'g': 0}}
{'color': {'a': 0, 'r': 255, 'b': 0, 'g': 0}}
{'color': {'a': 0, 'r': 0, 'b': 255, 'g': 0}}
{'color': {'a': 0, 'r': 0, 'b'

In [82]:
g=u'Specifically, Mandiant is providing the following:   Digital delivery of over 3,000 APT1 indicators, such as domain names, IP addresses, and MD5 hashes of   malware.'
g[53]

u'D'

In [83]:
from gensim import corpora,models,similarities
model.similarity('George', 'Mandiant')

KeyError: "word 'George' not in vocabulary"

In [84]:
sents[3]

u'Combined, the United  States and our allies in Europe and Asia have significant  diplomatic and economic leverage over China, and we should  use this to our advantage to put an end to this scourge.1  U.S. Rep. Mike Rogers, October, 2011  It is unprofessional and groundless to accuse  the Chinese military of launching cyber attacks  without any conclusive evidence.2  Chinese Defense Ministry, January, 2013 1  Mike Rogers, Statement to the U.S. House, Permanent Select Committee on Intelligence, Open Hearing: Cyber Threats and Ongoing Efforts to  Protect the Nation, Hearing, October 4, 2011, http://intelligence.house.gov/sites/intelligence.house.gov/files/documents/100411CyberHearingRogers.'

In [85]:
def tolist(sents):
    for sent in sents:
        yield list(sent)

lsents=tolist(sents)

In [86]:
model = gensim.models.Word2Vec(sents,size=1000)
print(model)
print(model.wv.vocab)

Word2Vec(vocab=84, size=1000, alpha=0.025)
{u'!': <gensim.models.keyedvectors.Vocab object at 0x15c8dcb90>, u' ': <gensim.models.keyedvectors.Vocab object at 0x118dae390>, u'%': <gensim.models.keyedvectors.Vocab object at 0x15c8dc390>, u'&': <gensim.models.keyedvectors.Vocab object at 0x154f17250>, u')': <gensim.models.keyedvectors.Vocab object at 0x15c8dc590>, u'(': <gensim.models.keyedvectors.Vocab object at 0x154fc2890>, u'+': <gensim.models.keyedvectors.Vocab object at 0x1137ce8d0>, u'-': <gensim.models.keyedvectors.Vocab object at 0x15c8dc450>, u',': <gensim.models.keyedvectors.Vocab object at 0x154fc2910>, u'/': <gensim.models.keyedvectors.Vocab object at 0x1137ceb10>, u'.': <gensim.models.keyedvectors.Vocab object at 0x154f17d50>, u'1': <gensim.models.keyedvectors.Vocab object at 0x15c8dce10>, u'0': <gensim.models.keyedvectors.Vocab object at 0x154fc2950>, u'3': <gensim.models.keyedvectors.Vocab object at 0x1137ce950>, u'2': <gensim.models.keyedvectors.Vocab object at 0x154f1741